In [1]:
import json
import os
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [2]:
agenda = """
'인공지능이 인간의 일자리를 위협합니다. 이에 대한 대응 방안을 논의합니다.'
"""


In [3]:
sampling_tempalte = """
{agenda}에 대해 논의 중이니다.
```
[이전 의견]:
{selected}
```
[이전 의견]에 대한 구체적이며 실질적인 구현 방안을 아래 JSON 형식으로 답하세요.
{{
    "주제": <주제>
    "구현": <50단어 이내로 작성하세요>,
    "근거": <[이전 의견]의 어떤 대목에서 그렇게 생각했는지>
 }}
"""


In [4]:
evaluation_template = """
{agenda}에 대해 논의하고 있습니다.
```
[의견]: 
{thought}
```
위의 의견]을 아래 JSON 형식으로 평가하세요.
{{
    "창의적이고 혁신적인 방법인가": <15점 만점 기준 점수>,
    "단기간 내에 실현 가능한 방법인지": <10점 만점 기준 점수>,
    "총점": <총점> 
}}
"""


In [5]:
def request_gpt(message, model, temperature, type="json_object"):
    message = [{"role": "user", "content": message}] 
    response = client.chat.completions.create(
                model=model,
                messages=message,
                temperature=temperature,
                response_format = {"type" : type}
            ).model_dump()
    if type == "json_object":
        response_content = json.loads(response['choices'][0]['message']['content'])
    else:
        response_content = response['choices'][0]['message']['content']
    return response_content


In [12]:
def generate_thoughts(selected):
    selected = "없음" if len(selected) == 0 else selected
    samples = []
    message = sampling_tempalte.format(agenda=agenda, selected=selected)
    for _ in range(5):
        # 필요하면 GPT-4로 대체해볼 것
        sample = request_gpt(message, "gpt-3.5-turbo-1106", temperature=1.2)
        samples.append(sample['구현'])
        #print("generate_thoughts:", sample['구현'])
    return samples


In [13]:
def evaluate(thoughts):
    values = []    
    for thought in thoughts:
        message = evaluation_template.format(agenda=agenda, thought=thought)
        value = request_gpt(message, "gpt-4-1106-preview", temperature=0)
        values.append({
            "thought": thought,
            "value": value
        })    
    return values

def get_top_n(values, n):
    return sorted(values, key=lambda x: x["value"]["총점"], reverse=True)[:n]


In [15]:
selected_list = []
selected = ""
for step in range(3):
    thoughts = generate_thoughts(selected)
    values = evaluate(thoughts)
    selected = get_top_n(values, 1)[0]['thought']
    selected_list.append(selected)
    print(f"{step + 1}단계: {selected}")

print("\n".join(selected_list))


1단계: 인간 채용을 통해 인공지능 시스템을 감시하고 향상시키는 업무를 만들어야 합니다.
2단계: 기업이 전문가를 고용하여, 인공지능 시스템의 감시 및 개선 업무를 담당하도록 합니다. 이를 통해 인간과 인공지능이 상호 보완적으로 작용하여 새로운 노동력을 창출합니다.
3단계: 전문가와 인공지능 시스템 간의 업무 분담을 효율적으로 조정하여, 전문가가 사람이 하기 어려운 복잡한 업무를 담당하고, 인공지능이 반복적이고 단순한 작업을 처리하도록 합니다.
인간 채용을 통해 인공지능 시스템을 감시하고 향상시키는 업무를 만들어야 합니다.
기업이 전문가를 고용하여, 인공지능 시스템의 감시 및 개선 업무를 담당하도록 합니다. 이를 통해 인간과 인공지능이 상호 보완적으로 작용하여 새로운 노동력을 창출합니다.
전문가와 인공지능 시스템 간의 업무 분담을 효율적으로 조정하여, 전문가가 사람이 하기 어려운 복잡한 업무를 담당하고, 인공지능이 반복적이고 단순한 작업을 처리하도록 합니다.


In [ ]:
summary = f"{agenda} 다음 내용을 근거로 짧은 보고서를 작성하세요:"+str(selected_list)
report = request_gpt(summary, "gpt-4", temperature=0, type="text")
print(report)


보고서: 인공지능의 일자리 위협에 대한 대응 방안

인공지능(AI)의 발전은 인간의 일자리를 위협하는 측면이 있습니다. 이에 대응하기 위한 방안으로, 교육 및 재직자를 위한 역량 강화 프로그램을 투입하는 것이 필요하다는 주장이 제기되고 있습니다. 이는 새로운 기술과 혁신에 대한 대비 능력을 키우고, 인공지능과 협업할 수 있는 역량을 갖출 수 있도록 지원하는 것을 목표로 합니다.

이를 위해 기업이나 정부는 인공지능과 협업하는 방법과 기술을 학습할 수 있는 교육 프로그램을 개설하고, 재직자들에게 참여를 장려해야 합니다. 이러한 교육 프로그램은 온라인 교육, 세미나, 워크샵 등 다양한 형태로 제공될 수 있습니다. 이를 통해 재직자들은 인공지능과의 협업 방법과 기술에 대한 이해를 높일 수 있습니다.

또한, 기업이나 정부는 인공지능과의 협업을 위한 온라인 교육 플랫폼을 제작하고, 재직자들을 대상으로 공개하여 참여를 유도해야 합니다. 이를 통해 재직자들은 실전에서 인공지능과의 협업 방법과 기술을 학습하게 됩니다.

결론적으로, 인공지능의 일자리 위협에 대응하기 위해서는 교육 및 재직자를 위한 역량 강화 프로그램의 투입이 필요하며, 이를 위해 기업이나 정부의 적극적인 지원이 요구됩니다. 이를 통해 인간은 인공지능과의 협업 능력을 키우고, 새로운 기술 변화에 대응할 수 있는 능력을 갖추게 될 것입니다.
